In [1]:
# !pip install spacy

from dotenv import load_dotenv
import ast

load_dotenv()

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/andriimyronenko/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
import pandas as pd

test_tech_df = pd.read_csv('./data/merged/test_df_tech_only.csv')

test_tech_df['knowledge'] = test_tech_df['knowledge'].apply(ast.literal_eval)
test_tech_df['skills'] = test_tech_df['skills'].apply(ast.literal_eval)

In [3]:
print(test_tech_df.at[0, 'text'])
print('>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>')
print(test_tech_df.at[0, 'skills'])
print('>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>')
print(test_tech_df.at[0, 'knowledge'])

Full Stack Software Engineer - Java / JavaScript <ORGANIZATION> <ORGANIZATION> <ORGANIZATION> <ORGANIZATION> . <ADDRESS> <ADDRESS> <LOCATION> - <LOCATION> Date posted: 2021-03-04 Likes: 0 Dislikes: 0 Love: 0 Salary: <SALARY> Job type: FULL_TIME Experience level: <EXPERIENCE> Industry: <INDUSTRY> Company size: <SIZE> Company type: <COMPANY_TYPE> Technologies: javascript reactjs java Job description: Job type: Full-time Role: Full Stack Developer Technologies javascript reactjs java Job description As a member of our <ORGANIZATION> <ORGANIZATION> <ORGANIZATION> we look first and foremost for people who are passionate around solving business problems through innovation and engineering practices . You'll be required to apply your depth of knowledge and expertise to all aspects of the software development lifecycle as well as partner continuously with your many stakeholders on a daily basis to stay focused on common goals . We embrace a culture of experimentation and constantly strive for i

In [4]:
verbose_template = """
You are a top-notch recruiter and data labeler.
Your task is to thoroughly analyze the following part of a job posting and extract text spans including skills or knowledge.

You should use the following guideline for annotation:

Instruction for Annotating Text for Skills and Knowledge Components
Objective: Identify and annotate spans of text in job postings (JPs) that represent specific skills or knowledge required for the position.

1. Identifying Skills:

General Rule: A skill is usually indicated by a VERB or an (ADJECTIVE) + NOUN combination.
Exclusion of Modal Verbs: Do not tag modal verbs (e.g., can, will) as part of the skill.
Phrase Separation: Split phrases containing prepositions and/or conjunctions, except when conjunctions coordinate two nouns as a single argument.
Handling Anaphoric Pronouns: Avoid tagging skills with anaphoric pronouns. Only tag the preceding skill.
Splitting Coordinated Components: Split nouns and adjectives in coordination if they lack a verb.
Listing of Skills: If skills are listed leading to different subtasks, annotate each separately.
Brevity in Contextual Information: Keep skill annotations concise, especially when followed by company-specific info.
Inclusion of 'skills' or 'knowledge' in Tags: Include these words in the annotation if their omission alters the meaning.
Parenthetical Information: Include if it elaborates on the skill or is an abbreviation.
Adverbial Inclusion: Include adverbials that describe the manner of doing something.
Attitudes as Skills: Annotate attitudes as skills, omitting articles.
Exclusions:
Avoid tagging ironic skills.
Avoid nested annotations; use one span for overlapping skills.
Do not tag skills in top headlines, but do tag in sub-headlines and body text.
Exclude fluff and triggers surrounding the skill component.
Exclude participation, contributing, and transfer expressions from skill annotations.
Do not annotate occupations or positions.
Focus on skills related and specific to the position.

2. Identifying Knowledge:
General Rule: Knowledge is non-executable and possessed by an individual.
Inclusion of Parenthetical Information: Include if it is related to the knowledge component.
Licenses and Certifications: Include additional words like "certificate," "license," etc.
Vague Preceding Verbs: Only tag the knowledge component if preceded by a vague verb.
Specificity: Annotate only specified knowledge components.
Nested Knowledge in Skills: Knowledge components can be nested within skill annotations.
Coordination of Knowledge Components: Annotate as one if all components share one knowledge tag.
Listing Knowledge Tags: Annotate all knowledge tags separately.
Annotation of Industries and Fields: Tag these as knowledge components.

3. Prioritization and Other Considerations:
Prefer skills over knowledge in uncertain cases.
Prioritize skills over attitudes; only tag the skill within an attitude.
Keep annotations concise and relevant to the job.
Annotate skills and knowledge in unconventional places if related to the position.
Consider annotating a combination of skill and knowledge when applicable.
Exclude knowledge/skill components in positions.
Focus on annotations relevant to the specific position and its future expectations.

Output just the JSON array, wihtout any other text. 
Start your response with "[" symbol, and finish with "]".
Make sure your response doesn't include anything else.

Job posting:
{job_posting}

Response:"""

short_template = """
    You are a top-notch recruiter and data labeler.
    Your task is to thoroughly analyze the following part of a job posting and extract text spans including skills and knowledge.
    A skill in a job context is typically indicated by a verb or an adjective-noun combination, reflecting an executable ability or a specific way of performing a task. 
    Knowledge, on the other hand, refers to non-executable information that an individual possesses, often indicated by specific fields, industries, or certifications, and is distinct from direct action or skills.

    Output the JSON array, wihtout any other text. 
    Start your response with "[" symbol, and finish with "]".
    Make sure your response doesn't include anything else.
        
    Job posting:
    {job_posting}
"""

few_shot_template = """
    You are a top-notch recruiter and data labeler.
    Your task is to thoroughly analyze the following part of a job posting and extract text spans including job-related skills or knowledge.
    A skill in a job context is typically indicated by a verb or an adjective-noun combination, reflecting an executable ability or a specific way of performing a task. 
    Knowledge, on the other hand, refers to non-executable information that an individual possesses, often indicated by specific fields, industries, or certifications, and is distinct from direct action or skills.
    
    Below you can see some examples. Pay attention to what is included in detected spans. We want not to include false positive skills and knowledge.

    Job posting:
    Ability to work in large collaborative teams to achieve organizational goals
    Detected spans:
    ["work in large collaborative teams"]

    Job posting:
    Work hands-on together with the other engineers within the Agile team 
    Detected spans:
    ["Work hands-on"]
    
    Job posting:
    Requirements At least 5 years of combined experience in Java or Kotlin and JavaScript or TypeScript programming and related test frameworks ( Selenium TestCafe etc.) .
    Detected spans:
    ["Java", "Kotlin", "JavaScript", "Typescript"]
    
    Job posting:
    A degree in Computer Science or related fields or equivalent practical experience . 
    Detected spans:
    ["degree in Computer Science"]
    
    Job posting:
    Experience in working on a cloud-based application running on Docker .
    Detected spans:
    ["Docker", "working on a cloud-based application"]
    
    Make sure to never include protected attributes like gender identity or veteran status in the returned list:
    
    Job posting:
    We do not discriminate on the basis of any protected attribute including race religion color national origin gender sexual orientation .
    Detected spans:
    []

    Output just the JSON array, wihtout any other text. 
    Start your response with "[" symbol, and finish with "]".
    Make sure your response doesn't include data from examples and anything else except from JSON array.
    Return empty array If there are no obvious skill and knowledge spans in the given posting.
    
    Job posting:
    {job_posting}
    Detected spans:
"""

annotation_guidelines_template = """
    You are a top-notch recruiter and data labeler.
    Your task is to thoroughly analyze the following part of a job posting and extract text spans including candidate-related skill and knowledge components.
    The definition of skill component is the following:
    "The ability to apply knowledge and use know-how to complete tasks and solve problems. Skills are described as cognitive (involving the use of logical, intuitive and creative thinking) or practical (involving manual dexterity and the use of methods, materials, tools and instruments)."
    The definition of knowledge is the following:
    "The body of facts, principles, theories and practices that is related to a field of work or study. Knowledge is described as theoretical and/or factual, and is the outcome of the assimilation of information through learning."
    Make sure to detect the shortest possible text span excluding irrelevant, company-specific information. 
    
    Output just the JSON array, with skill or knowledge strings surrounded by quotation marks. 
    Start your response with "[" symbol, and finish with "]".
    
    Example job posting including skill components:
    We are looking for people who want to work in cross-functional teams and want to help others learn how to build them    
    Detected spans:
    ["work in cross-functional teams", "help others learn"]
    
    Example job posting including knowledge components:
    Experience with latest versions of Java and C++.
    Detected knowledge components:
    ["Java", "C++"]
    
    Make sure your response doesn't include anything else except from JSON array. Each skill should be a separate string.
    Return empty array If there are no obvious skill or knowledge components in the given posting.
    
    Job posting:
    {job_posting}
    Detected spans:
"""


template_collection = {
    'verbose': verbose_template,
    'short': short_template,
    'few_shot': few_shot_template,
    'annotation_guidelines_template': annotation_guidelines_template
}

In [5]:
from langchain import LLMChain, PromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

prompt_template = PromptTemplate(input_variables=["job_posting"], template=template_collection['few_shot'])

In [12]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("user", "{input}")
])

In [13]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

chain = prompt | llm | output_parser

In [14]:
import ast

test_posting = test_tech_df.at[0, 'text']

def get_skills_for_posting(job_posting):
    prompt_val = prompt_template.invoke({"job_posting": job_posting})
    output = chain.invoke({"input": prompt_val.to_string()})
    array_start = output.find('[')
    array_end = output.find(']', array_start)
    array_string = output[array_start:array_end+1].strip()
    return ast.literal_eval(array_string)
     
gpt_results = get_skills_for_posting(test_posting)
gpt_results

['Java',
 'JavaScript',
 'knowledge of application data and infrastructure architecture disciplines',
 'architecture and design',
 'developmental toolsets',
 'industry-wide technology strategies and best practices',
 'large collaborative teams',
 'building an innovative culture',
 'business analysis development maintenance and software improvement',
 'HTML5 / JavaScript',
 'developing multithreaded scalable concurrent applications',
 'Node JS',
 'React or similar view framework']

In [15]:
def calculate_f1_score(real_values, predicted_values):
    real_values = [val.lower() for val in real_values]
    predicted_values = [val.lower() for val in predicted_values]
 
    tp = sum([1 for val in predicted_values if val in real_values])
    fp = sum([1 for val in predicted_values if val not in real_values])
    fn = sum([1 for val in real_values if val not in predicted_values])

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    print("Precision: " + str(precision))
    print("Recall: " + str(recall))

    return f1_score

In [16]:
merged_skills_knowledge = test_tech_df.at[0, 'skills'] + test_tech_df.at[0, 'knowledge']

In [17]:
gpt_results

['Java',
 'JavaScript',
 'knowledge of application data and infrastructure architecture disciplines',
 'architecture and design',
 'developmental toolsets',
 'industry-wide technology strategies and best practices',
 'large collaborative teams',
 'building an innovative culture',
 'business analysis development maintenance and software improvement',
 'HTML5 / JavaScript',
 'developing multithreaded scalable concurrent applications',
 'Node JS',
 'React or similar view framework']

In [18]:
calculate_f1_score(merged_skills_knowledge, gpt_results)

Precision: 0.46153846153846156
Recall: 0.20689655172413793


0.28571428571428575

In [19]:
full_text_predictions_f1_scores = []

for index, row in test_tech_df.iterrows():
    try:
        row_merged_skills_knowledge = test_tech_df.at[index, 'skills'] + test_tech_df.at[index, 'knowledge']
        res = get_skills_for_posting(row['text'])
        test_tech_df.at[index, "predictions_gpt_full_text"] = res
        f1_score = calculate_f1_score(row_merged_skills_knowledge, res)
        full_text_predictions_f1_scores.append(f1_score)
    except Exception as e:
        print(f"Failed to parse skills for row {index}: {e}")
        
full_text_predictions_f1_scores

Failed to parse skills for row 0: Must have equal len keys and value when setting with an iterable
Precision: 0.5833333333333334
Recall: 0.23333333333333334
Precision: 0.43333333333333335
Recall: 0.4126984126984127
Failed to parse skills for row 3: invalid syntax (<unknown>, line 0)
Precision: 0.8095238095238095
Recall: 0.7727272727272727
Precision: 0.7692307692307693
Recall: 0.2564102564102564
Precision: 0.19444444444444445
Recall: 0.3888888888888889
Precision: 0.7647058823529411
Recall: 0.25
Precision: 0.43478260869565216
Recall: 0.5263157894736842
Precision: 0.16666666666666666
Recall: 0.0975609756097561
Failed to parse skills for row 10: invalid syntax (<unknown>, line 0)
Precision: 0.43478260869565216
Recall: 0.2857142857142857
Precision: 0.125
Recall: 0.02631578947368421
Precision: 0.25
Recall: 0.15789473684210525
Precision: 0.5
Recall: 0.22727272727272727
Precision: 0.25806451612903225
Recall: 0.37209302325581395
Precision: 0.7692307692307693
Recall: 0.3125
Precision: 0.075
Reca

[0.33333333333333337,
 0.4227642276422764,
 0.7906976744186046,
 0.38461538461538464,
 0.25925925925925924,
 0.37681159420289856,
 0.47619047619047616,
 0.12307692307692307,
 0.3448275862068965,
 0.043478260869565216,
 0.1935483870967742,
 0.3125,
 0.30476190476190473,
 0.4444444444444445,
 0.11428571428571428,
 0.4482758620689655,
 0.4150943396226416,
 0.5777777777777778,
 0.1276595744680851,
 0.2278481012658228,
 0.32142857142857145,
 0.15384615384615383,
 0.32653061224489793,
 0.3673469387755102,
 0.5750000000000001,
 0.2692307692307693,
 0.34782608695652173,
 0.3157894736842105]

In [23]:
sum(full_text_predictions_f1_scores) / len(full_text_predictions_f1_scores)

0.33565176542051367